In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Mar  6 17:50:15 2020

@author: anselhsu
"""

import sys
import os
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
import functools
import numpy as np
import random as rd
import matplotlib
import time
matplotlib.use("Qt5Agg")
from matplotlib.figure import Figure
from matplotlib.animation import TimedAnimation
from matplotlib.lines import Line2D
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import datetime
import threading
import serial
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import pandas as pd

#os.chdir(r'D:\research\ui test')

class CustomMainWindow(QMainWindow):
    def __init__(self):
        super(CustomMainWindow, self).__init__()
        # Define the geometry of the main window
        self.l_model = ''
        self.xy = self.grid_create()
        self.setGeometry(60, 100, 1400, 900)
        self.setWindowTitle("Abnormal Detect System")
        # Create FRAME_A
        self.FRAME_A = QFrame(self)
        self.FRAME_A.setStyleSheet("QWidget { background-color: %s }" % QColor(214, 234, 248,255).name())
        self.LAYOUT_A = QGridLayout()
        self.FRAME_A.setLayout(self.LAYOUT_A)
        self.setCentralWidget(self.FRAME_A)
#        self.LAYOUT_A.setRowStretch(1, 1)
        self.LAYOUT_A.setRowStretch(2, 1)
        self.LAYOUT_A.setRowStretch(3, 1)
#        self.LAYOUT_A.setColumnStretch(2, 6)
        self.styleLabel = QLabel("Real time information board")
        self.styleLabel.setFont(QFont("Arial", 16))
        self.LAYOUT_A.addWidget(self.styleLabel,*(0,0,1,4,Qt.AlignCenter))
        self.palette = QPalette()
        self.palette.setColor(QPalette.Foreground, QColor(40, 116, 166, 255))
        self.styleLabel.setPalette(self.palette)
        
        # Place the zoom button
        self.DataBox = QGroupBox("Data")
        self.layout_B1 = QVBoxLayout()
        self.ModelBtn = QPushButton(text = 'Data collect')
        self.ModelBtn.setFixedSize(150, 50)
#        self.ModelBtn.clicked.connect(self.save_btn)
        self.ModelBtn2 = QPushButton(text = 'Link Data')
        self.ModelBtn2.setFixedSize(150, 50)
        self.ModelBtn2.clicked.connect(self.linkAction)
        self.layout_B1.addWidget(self.ModelBtn)
        self.layout_B1.addWidget(self.ModelBtn2)
        self.DataBox.setLayout(self.layout_B1)
#
        self.LAYOUT_A.addWidget(self.DataBox, *(4,2,1,1,Qt.AlignCenter))
        # Place the matplotlib figure
        self.myFig = CustomFigCanvas(upb=60,lwb=15)
        self.styleLabel1 = QLabel("溫度")
        self.LAYOUT_A.addWidget(self.styleLabel1,*(1,0,1,1,Qt.AlignCenter))
        self.LAYOUT_A.addWidget(self.myFig, *(2,0,1,1))
        
        self.styleLabel1 = QLabel("震動")
        self.LAYOUT_A.addWidget(self.styleLabel1,*(1,1,1,1,Qt.AlignCenter))
        self.myFig2 = CustomFigCanvas(upb=1.5,lwb=0)
        self.LAYOUT_A.addWidget(self.myFig2, *(2,1,1,1))
        
        self.styleLabel1 = QLabel("噪音")
        self.LAYOUT_A.addWidget(self.styleLabel1,*(1,2,1,1,Qt.AlignCenter))
        self.myFig3 = CustomFigCanvas(upb=100,lwb=50)
        self.LAYOUT_A.addWidget(self.myFig3, *(2,2,1,1))
        
        self.myFig4 = CustomFigCanvas(upb=1,lwb=0)
        self.LAYOUT_A.addWidget(self.myFig4, *(3,2,1,1))
        
        self.myFig5 = CustomFigCanvas(upb=100,lwb=0)
        self.LAYOUT_A.addWidget(self.myFig5, *(3,0,2,2))
        
        # Add the callbackfunc to ..
        myDataLoop = threading.Thread(name = 'myDataLoop', target = dataSendLoop, daemon = True, args = (self.addData_callbackFunc,))
        myDataLoop.start()
        self.show()
        return

    def zoomBtnAction(self):
        if self.myFig.detect != 'open' and self.myFig.l_model == 'ready':
            self.listWidget.insertItem(0,"Detection start.")
            self.myFig.detect = 'open'
        else:
            if self.myFig.l_model != 'ready':
                self.listWidget.insertItem(0,"No model loaded.")
        return
    def closeAction(self):
        if self.myFig.detect != '':
            self.listWidget.insertItem(0,"Detection close.")
            self.myFig.detect = ''
            self.myFig.seri = []
            self.myFig.pd_ind = 0
        return
    def train_btn(self):
        if self.myFig.detect == 'open' or self.myFig.l_model == 'ready':
            if self.myFig.l_model == 'ready':
                self.listWidget.insertItem(0,"Model exist.")
            else:
                self.listWidget.insertItem(0,"Please close detection first.")
        else:
            self.listWidget.insertItem(0,"Training start.")
            self.train_dat = np.load('dataset/train_dat.npy' ,allow_pickle=True)
            self.train_dat = self.train_dat.item()
            self.train_dat = self.myFig.dat_trans(self.train_dat['array'],10)
            f_kde , f_pca , f_final_v ,f_umap = self.train_func(self.train_dat)
            self.listWidget.insertItem(0,"Training finished.")
            joblib.dump(f_umap, 'model/u_m.sav')
            joblib.dump(f_pca, 'model/p_m.sav')
            joblib.dump(f_kde, 'model/dp_m.sav')
            np.save('model/related_value.npy',f_final_v)
            self.listWidget.insertItem(0,"Model saved.")
        return
    def loadm_btn(self):
        if self.myFig.l_model == '':
            self.listWidget.insertItem(0,"Load model.")
            self.myFig.model_p = joblib.load('model/p_m.sav')
            self.myFig.model_u = joblib.load('model/u_m.sav')
            self.myFig.model_pd = joblib.load('model/dp_m.sav')
            m_para = np.load('model/related_value.npy' ,allow_pickle=True)
            self.myFig.de_para = m_para.item()
            self.myFig.l_model = 'ready'
        else:
            self.listWidget.insertItem(0,"Load model already.")
            
            
    def check_btn(self):
        if self.myFig.now_cum>999 and self.myFig.l_model == 'ready':
            self.listWidget.insertItem(0,"Start checking.")
            self.check_func()
        else:
            if self.myFig.l_model != 'ready':
                self.listWidget.insertItem(0,"No model loaded.")
            else:
                self.listWidget.insertItem(0,"Data not enough.")
        return
    
    def grid_create(self,c_num = 15):
        xgrid , ygrid = np.arange(0,1,1/c_num) , np.arange(0,1,1/c_num)
        X, Y = np.meshgrid(xgrid, ygrid[::-1])
        xy = np.vstack([X.ravel(), Y.ravel()]).T
        return xy
    
    def check_func(self,bd = 0.05):
        ch_dat = self.myFig.now_dat_array
        ch_dct = self.myFig.dat_trans(ch_dat,10)
        e_n = np.array(self.myFig.model_u.transform(ch_dct))
        e_n[:,0] = e_n[:,0]-self.myFig.de_para['ux_min']
        e_n[:,0] = e_n[:,0]/self.myFig.de_para['ux_max']
        e_n[:,1] = e_n[:,1]-self.myFig.de_para['uy_min']
        e_n[:,1] = e_n[:,1]/self.myFig.de_para['uy_max']
        l1 = np.logical_and(e_n[:,0]>=0 , e_n[:,0]<=1)
        l2 = np.logical_and(e_n[:,1]>=0 , e_n[:,1]<=1)
        lr = sum(np.logical_and(l1,l2))/len(e_n)
        kde_2 = KernelDensity(bandwidth=bd, metric='haversine',kernel='gaussian', algorithm='ball_tree')
        kde_2.fit(e_n)
        Zn = np.exp(kde_2.score_samples(self.xy))
        Zn = (lr*Zn/sum(Zn))+(1-lr)/225
        KL_value = sum(Zn*np.log(Zn/self.myFig.de_para['UMAP_p']))
        self.listWidget.insertItem(0,"KL- divergence is "+str(round(KL_value,3))+".")
        if KL_value>1:
            self.listWidget.insertItem(0,"Transfer exist.")
        return
    
    def save_btn(self):
        if self.myFig.data_switch != 'open':
            self.listWidget.insertItem(0,"Starting collect data.")
            self.myFig.data_switch = 'open'
        return
    def linkAction(self):
        if self.myFig.link != 'open':
#            self.listWidget.insertItem(0,"Link data.")
            self.myFig.link = 'open'
            self.myFig2.link = 'open'
            self.myFig3.link = 'open'
            self.myFig4.link = 'open'
        return
    def addData_callbackFunc(self, value):
        self.myFig.addData(value[0])
        self.myFig2.addData(value[1])
        self.myFig3.addData(value[2])
        self.myFig4.addData(value[3])
        return
    
    def train_func(self,dat,bd = 0.05):
        pca = PCA(n_components=2)
        pca_res = pca.fit_transform(dat)
        train_x_min ,train_y_min = min(pca_res[:,0]) , min(pca_res[:,1])
        pca_res[:,0] = pca_res[:,0]-train_x_min
        pca_res[:,1] = pca_res[:,1]-train_y_min
        train_x_max , train_y_max = max(pca_res[:,0]) , max(pca_res[:,1])
        pca_res[:,0] = pca_res[:,0]/train_x_max
        pca_res[:,1] = pca_res[:,1]/train_y_max
        
        umap_reducer = umap.UMAP()
        umap_reducer.fit(dat)
        umap_res = np.array(umap_reducer.embedding_)
        u_x_min ,u_y_min = min(umap_res[:,0]) , min(umap_res[:,1])
        umap_res[:,0] = umap_res[:,0]-u_x_min
        umap_res[:,1] = umap_res[:,1]-u_y_min
        u_x_max , u_y_max = max(umap_res[:,0]) , max(umap_res[:,1])
        umap_res[:,0] = umap_res[:,0]/u_x_max
        umap_res[:,1] = umap_res[:,1]/u_y_max
        
        kde_u = KernelDensity(bandwidth=bd, metric='haversine',kernel='gaussian', algorithm='ball_tree')
        kde_u.fit(umap_res)
        Z = np.exp(kde_u.score_samples(self.xy))
        Z = Z/sum(Z)
        
        kde = KernelDensity(bandwidth=bd, metric='haversine',kernel='gaussian', algorithm='ball_tree')
        kde.fit(pca_res)
        ori_est = np.exp(kde.score_samples(pca_res))
        threshold = 2*np.mean(ori_est)/self.myFig.collect_dim
        final_v = {'x_min':train_x_min,'y_min':train_y_min,
                   'x_max':train_x_max,'y_max':train_y_max,
                   'ux_min':u_x_min,'uy_min':u_y_min,
                   'ux_max':u_x_max,'uy_max':u_y_max,
                   'thres':threshold,'UMAP_p':Z}
        return kde , pca , final_v , umap_reducer

''' End Class '''


class CustomFigCanvas(FigureCanvas, TimedAnimation):
    def __init__(self,upb=1000,lwb=0):
        self.model_p , self.model_pd , self.de_para ,self.l_model ,self.model_u = '','','','',''
        self.addedData , self.condData , self.data_npy = [],[],[]
        self.detect , self.link , self.data_switch , self.check = '','','',''
        self.seri , self.y_hyp ,self.y_lwp = [] , upb , lwb
        # The data
        self.xlim = 200
        self.collect_dim = 10 # unit dim
        self.now_dat_array , self.now_cum = np.zeros(1000) , 0
        self.n = np.linspace(0, self.xlim - 1, self.xlim)
        self.y_dist = self.y_hyp - self.y_lwp
        self.ynd_d = (self.y_dist/3)+self.y_lwp
        self.ynd_p = (self.y_dist/2)+self.y_lwp
        self.y = (self.n * 0.0) + self.ynd_d
        self.y2 = (self.n * 0.0) + self.ynd_p
        # The window
        self.fig = Figure(figsize=(5,5), dpi=100)
        self.ax1 = self.fig.add_subplot(111)
        # self.ax1 settings
        self.ax1.set_xlabel('time')
        self.ax1.set_ylabel('raw data')
        self.line1 = Line2D([], [], color='#A9CCE3')
        self.ax1.add_line(self.line1)
        self.ax1.axvline(x=180, color='#ABEBC6')
        self.ax1.set_xlim(0, self.xlim - 1)
        self.ax1.set_ylim(self.y_lwp, self.y_hyp)
        
        FigureCanvas.__init__(self, self.fig)
        TimedAnimation.__init__(self, self.fig, interval = 50, blit = True)
        return

    def new_frame_seq(self):
        return iter(range(self.n.size))

    def _init_draw(self):
        lines = [self.line1]
        for l in lines:
            l.set_data([],[])
        return
    
    def addData(self, value):
        global myGUI
        if self.data_switch == 'open':
            if len(self.data_npy)< 10*self.collect_dim :
                self.data_npy.append(value)
            else:
                np.save('dataset/train_dat.npy',{'array':self.data_npy,'time':time.ctime()})
                self.data_npy= []
                self.data_switch = ''
                myGUI.listWidget.insertItem(0,"Finish collect and save.")
            if len(self.data_npy)/self.collect_dim in np.arange(1,10):
                myGUI.listWidget.insertItem(0,str(len(self.data_npy)*10/self.collect_dim)+"% complete.")
        if self.link == 'open':
            self.addedData.append(value)
            self.now_dat_array = np.roll(self.now_dat_array, -1)
            self.now_dat_array[-1] = value
            if self.now_cum<1000:
                self.now_cum += 1
        else:
            self.addedData.append(self.ynd_d)
        if self.detect == 'open':
            if self.l_model == 'ready':
                if len(self.seri)>9:
                    self.det_func(self.seri)
                    self.seri = []
                else:
                    self.seri.append(value)
        else:
            if len(self.condData)>0:
                del(self.condData[0])
            self.condData.append(self.ynd_p)
        return
    
    def dat_trans(self,x,l):
        d_len = int(len(x)/l)
        new_dat = np.zeros((d_len,l))
        for i in range(d_len):
            new_dat[i,:] = x[i*10:(i*10)+10]
        return dct(new_dat)[:,:5]
    def det_func(self,dat):
        t1 = dct(dat)[np.newaxis,:5]
        x_res = self.model_p.transform(t1)
        x_res[:,0] = x_res[:,0]-self.de_para['x_min']
        x_res[:,0] = x_res[:,0]/self.de_para['x_max']
        x_res[:,1] = x_res[:,1]-self.de_para['y_min']
        x_res[:,1] = x_res[:,1]/self.de_para['y_max']
        f_r = int(np.exp(self.model_pd.score_samples(x_res))<self.de_para['thres'])
        self.condData.extend([(self.y_dist*0.98)*f_r+(self.y_lwp+(self.y_dist*0.01))] *11)
        return
    def zoomIn(self, value):
        bottom = self.ax1.get_ylim()[0]
        top = self.ax1.get_ylim()[1]
        bottom += value
        top -= value
        self.ax1.set_ylim(bottom,top)
        self.draw()
        return

    def _step(self, *args):
        try:
            TimedAnimation._step(self, *args)
        except Exception as e:
            self.abc += 1
            print(str(self.abc))
            TimedAnimation._stop(self)
            pass
        return
    
    def _draw_frame(self, framedata):
        while(len(self.addedData) > 0):
            self.y = np.roll(self.y, -1)
            self.y[-1] = self.addedData[0]
            self.y2 = np.roll(self.y2, -1)
            self.y2[-1] = self.ynd_p
            del(self.addedData[0])
            if len(self.seri)>9:
                self.y2[-11:] = self.condData[-11:]
                self.condData = []
        self.line1.set_data(self.n[ 0 : self.n.size ], self.y[ 0 : self.n.size ])
        self._drawn_artists = [self.line1]
        return

''' End Class '''

# You need to setup a signal slot mechanism, to
# send data to your GUI in a thread-safe way.
# Believe me, if you don't do this right, things
# go very very wrong..
class Communicate(QObject):
    data_signal = pyqtSignal(tuple)

''' End Class '''
def dataSendLoop(addData_callbackFunc):
    # Setup the signal-slot mechanism.
    mySrc = Communicate()
    mySrc.data_signal.connect(addData_callbackFunc)
    L = []
    while(True):
        try:
            rr1 = master.execute(0x01, cst.READ_HOLDING_REGISTERS, 0x00 , 3)
            time.sleep(0.15)
            rr2 = master.execute(0x02, cst.READ_HOLDING_REGISTERS, 0x03F0 , 1)
            mySrc.data_signal.emit((rr1[0]*400/65535,rr1[1]*20/65535,rr1[2]*130/65535,rr2[0]/1000))
            L.append(rr1[1])
            if len(L)>=200:
                L = np.array(L)
                np.save('vib_{}.npy'.format(datetime.datetime.now().strftime("%m_%d_%H_%M_%S")), L)
                L = []
        except Exception as e:
            continue
        time.sleep(0.1)
   
    ###
###

# 1,4 use threshold

if __name__== '__main__':
    mbComPort = 'COM3'   # your RS-485 port. for linux --> "/dev/ttyUSB2"
    baudrate = 9600
    databit = 8 #7, 8
    parity = 'N' #N, O, E
    stopbit = 1 #1, 2
    mbTimeout = 100 # ms

    mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
    master = modbus_rtu.RtuMaster(mb_port)
    master.set_timeout(mbTimeout/1000.0)
    
    app = QApplication(sys.argv)
    QApplication.setStyle(QStyleFactory.create('Plastique'))
    myGUI = CustomMainWindow()
    sys.exit(app.exec_())
    master._do_close()

SystemExit: 0

C:\Users\MH630\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
master._do_close()

True